# Projeto 1 - Ciência dos Dados

Nome: Vinícius de Barros Gomes

Nome: Gabriel Nakano Fujihara

Nome: Pedro Roberto Fernandes Noronha

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\dev\DataScienceProject


Carregando a base de dados com os tweets classificados manualmente:

In [4]:
train = pd.read_excel('dados_treino.xlsx')
train.head(5)

Mensagem  \
0  Não li, até pq não consegui baixar. Vcs podem ...   
1  Não tem nada indicando que é em português de P...   
2  Produto horrível, veio com várias páginas em b...   
3                      O minimo é não ler esse livro   
4     Parece que o autor não leu o próprio livro!! 🙄   

  Target (neutro, positivo, negativo)  
0                              neutro  
1                              neutro  
2                              neutro  
3                            negativo  
4                            negativo

In [5]:
test = pd.read_excel('dados_teste.xlsx')
test.head(5)

Mensagem  \
0  Apesar da narrativa prender o leitor, os argum...   
1  A formatação desse ebook atrapalha muito a lei...   
2  Não é o primeiro livro da Record que oxida em ...   
3  O hype que envolve esse livro é ENORME então e...   
4  meu deus, essa escritora conseguiu estragar um...   

  Target (Neutro, Positivo, Negativo)  
0                            negativo  
1                              neutro  
2                              neutro  
3                            negativo  
4                            negativo

___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação dos tweets (Target).

Buscamos avaliar se os livros oferecidos pela amazon são ou não bons, portanto, foi ignorado quando haviam críticas exclusivamente ao serviço da amazon, como entrega e ou qualidade física do produto.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [6]:
def filtrador(df):
    comentarios = df["Mensagem"]
    data_frame =pd.DataFrame({})
    for i in comentarios:
        frase = i.lower()
        frase_lista = list(frase.split("."))
        frase_final=""
        for e in frase_lista:
            u = list(e.split(","))
            for w in u:
                frase_final+=w
                frase_final+=" "
        df2 = pd.DataFrame({"Mensagem": [frase_final]})
        data_frame = pd.concat([data_frame, df2], ignore_index=True)
    return data_frame
TESTE = filtrador(test)
TESTE.head(6)

Mensagem
0  apesar da narrativa prender o leitor  os argum...
1  a formatação desse ebook atrapalha muito a lei...
2  não é o primeiro livro da record que oxida em ...
3  o hype que envolve esse livro é enorme então e...
4  meu deus  essa escritora conseguiu estragar um...
5  o conteúdo é interessante mas está parecendo q...

In [7]:
def filtrador(df):
    comentarios = df["Mensagem"]
    data_frame =pd.DataFrame({})
    for i in comentarios:
        frase = i.lower()
        frase_lista = list(frase.split("."))
        frase_final=""
        for e in frase_lista:
            u = list(e.split(","))
            for w in u:
                frase_final+=w
                frase_final+=" "
        df2 = pd.DataFrame({"Mensagem": [frase_final]})
        data_frame = pd.concat([data_frame, df2], ignore_index=True)
    return data_frame
TESTE = filtrador(test)
TESTE.head(6)
def classificador(df):
    atraso=["não chegou", "atrasado", "nem chegou","demorou"]
    fisico=["rago","ragado","amassado","amasso","embalagem","embalado"]
    kindle=["kindle","ebook","ebok","e-book","e book","biblioteca"]
    preco=["impostos","caro","preço"," preco "]
    software=["site","aplicativo"]
    comentarios=df['Mensagem']
    df2 = pd.DataFrame(columns=["Target"])
    quantidade={"atraso":0,"kindle":0,"físico":0,"preço":0,"software":0}
    lista_final=[]
    for i in comentarios:
        atrasado=False
        destruido=False
        ebook=False
        valor=False
        softwar3=False
        lista=[]
        for e in i.split(" "):
            if e in atraso:
                atrasado=True
            if e in fisico:
                destruido=True
            if e in kindle:
                ebook=True
            if e in preco:
                valor=True
            if e in software:
                softwar3=True
            if atrasado:
                quantidade["atraso"]+=1
                if "Atraso" not in lista:
                    lista.append("Atraso")
            if destruido:
                quantidade["físico"]+=1
                if "Físico" not in lista:
                    lista.append("Físico")
            if softwar3:
                quantidade["software"]+=1
                if "Software" not in lista:
                    lista.append("Software")
            if ebook:
                quantidade["kindle"]+=1
                if "Kindle" not in lista:
                    lista.append("Kindle")
            if valor:
                quantidade["preço"]+=1
                if "Preço" not in lista:
                    lista.append("Preço")
            classificacao=""
        for y in range(len(lista)):
            if y<len(lista)-1:
                classificacao+=lista[y]+", "
            else:
                classificacao+=lista[y]
        if classificacao=="":
            classificacao+="Indefinido"
        lista_final.append(classificacao)
    df["Target"]=lista_final
    print(df)
    total=0
    for i in quantidade.values():
        total+=i
    probabilidade={}
    for i in quantidade.keys():
        probabilidade[i]=quantidade[i]/total
    print(probabilidade)
    return lista
classificador(TESTE)

                                              Mensagem      Target
0    apesar da narrativa prender o leitor  os argum...  Indefinido
1    a formatação desse ebook atrapalha muito a lei...      Kindle
2    não é o primeiro livro da record que oxida em ...  Indefinido
3    o hype que envolve esse livro é enorme então e...  Indefinido
4    meu deus  essa escritora conseguiu estragar um...  Indefinido
..                                                 ...         ...
245  <a class="a-link-normal" data-hook="product-li...       Preço
246  o livro tem muitas coisas boas sobre o assunto...  Indefinido
247  infelizmente a história é fraca e a escrita de...  Indefinido
248  ao longo do livro me senti extremamente mal ao...  Indefinido
249  apesar do enorme apreço que tenho por mário s ...  Indefinido

[250 rows x 2 columns]
{'atraso': 0.0, 'kindle': 0.4222427318874019, 'físico': 0.06091370558375635, 'preço': 0.21873557914167052, 'software': 0.2981079833871712}


[]

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**